In [55]:
def print_result(predict_label, id2rel, start_idx=8001):
    with open('predicted_result.txt', 'w', encoding='utf-8') as fw:
        for i in range(0, predict_label.shape[0]):
            fw.write('{}\t{}\n'.format(
                start_idx+i, id2rel[int(predict_label[i])]))

In [56]:
def train(model, criterion, loader, config):
    train_loader, dev_loader, _ = loader
    optimizer = optim.Adam(model.parameters(), lr=config.lr,
                           weight_decay=config.L2_decay)

    print(model)
    print('traning model parameters:')
    for name, param in model.named_parameters():
        if param.requires_grad:
            print('%s :  %s' % (name, str(param.data.shape)))
    print('--------------------------------------')
    print('start to train the model ...')

    eval_tool = Eval(config)
    max_f1 = -float('inf')
    for epoch in range(1, config.epoch+1):
        for step, (data, label) in enumerate(train_loader):
            model.train()
            data = data.to(config.device)
            label = label.to(config.device)

            optimizer.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            optimizer.step()

        _, train_loss, _ = eval_tool.evaluate(model, criterion, train_loader)
        f1, dev_loss, _ = eval_tool.evaluate(model, criterion, dev_loader)

        print('[%03d] train_loss: %.3f | dev_loss: %.3f | micro f1 on dev: %.4f'
              % (epoch, train_loss, dev_loss, f1), end=' ')
        if f1 > max_f1:
            max_f1 = f1
            torch.save(model.state_dict(), os.path.join(
                config.model_dir, 'model.pkl'))
            print('>>> save models!')
        else:
            print()


In [57]:
def test(model, criterion, loader, config):
    print('--------------------------------------')
    print('start test ...')

    _, _, test_loader = loader
    model.load_state_dict(torch.load(
        os.path.join(config.model_dir, 'model.pkl')))
    eval_tool = Eval(config)
    f1, test_loss, predict_label = eval_tool.evaluate(
        model, criterion, test_loader)
    print('test_loss: %.3f | micro f1 on test:  %.4f' % (test_loss, f1))
    return predict_label


In [58]:
#train dataset simeval2010

In [61]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from config import Config
from utils import WordEmbeddingLoader, RelationLoader, SemEvalDataLoader
from model import CNN
from evaluate import Eval
config = Config()
print('--------------------------------------')
print('some config:')
config.print_config()

print('--------------------------------------')
print('start to load data ...')
word2id, word_vec = WordEmbeddingLoader(config).load_embedding()
rel2id, id2rel, class_num = RelationLoader(config).get_relation()
loader = SemEvalDataLoader(rel2id, word2id, config)

train_loader, dev_loader = None, None
if config.mode == 1:  # train mode
    train_loader = loader.get_train()
    dev_loader = loader.get_dev()
test_loader = loader.get_test()
loader = [train_loader, dev_loader, test_loader]
print('finish!')

print('--------------------------------------')
model = CNN(word_vec=word_vec, class_num=class_num, config=config)
model = model.to(config.device)
criterion = nn.CrossEntropyLoss()

if config.mode == 1:  # train mode
    train(model, criterion, loader, config)
predict_label = test(model, criterion, loader, config)
print_result(predict_label, id2rel)


--------------------------------------
some config:
data_dir = ./data
output_dir = ./output
embedding_path = ./embedding/hlbl-embeddings-scaled.EMBEDDING_SIZE=50.txt
word_dim = 50
model_name = CNN
mode = 1
seed = 5782
cuda = -1
epoch = 20
dropout = 0.5
batch_size = 128
lr = 0.001
max_len = 100
pos_dis = 50
pos_dim = 5
hidden_size = 100
filter_num = 200
window = 3
L2_decay = 1e-05
device = cpu
model_dir = ./output/CNN
--------------------------------------
start to load data ...
finish!
--------------------------------------
CNN(
  (word_embedding): Embedding(246123, 50)
  (pos1_embedding): Embedding(103, 5)
  (pos2_embedding): Embedding(103, 5)
  (conv): Conv2d(1, 200, kernel_size=(3, 60), stride=(1, 1), padding=(1, 0))
  (maxpool): MaxPool2d(kernel_size=(100, 1), stride=(100, 1), padding=0, dilation=1, ceil_mode=False)
  (tanh): Tanh()
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=200, out_features=100, bias=True)
  (dense): Linear(in_features=100, out_feat